In [8]:
import requests
from urllib.parse import quote
import base64
import time
from datetime import datetime
import json

In [20]:
class Ddwrt:
    def __init__(self, username, password, host="192.168.0.1"):
        self._username = username
        self._password = password
        self._host = host

    def _get_headers(self):
        headers = {
            'Referer': 'http://%s/Status_Wireless.asp' % self._host,
            'Authorization': self._auth_header(),
        }
        response = requests.get('http://192.168.66.4/Status_Wireless.asp')
        return response

    def _auth_header(self):
        auth_bytes = bytes(self._username+':'+self._password, 'utf-8')
        auth_b64_bytes = base64.b64encode(auth_bytes)
        auth_b64_str = str(auth_b64_bytes, 'utf-8')

        auth_str = quote('Basic {}'.format(auth_b64_str))
        return auth_str
    


In [21]:
bf = Ddwrt("casti", "AmE3024OT1esVlj63", "192.168.66.4")

In [22]:
bf._get_headers()

<Response [401]>

In [7]:
import json
print(response.status_code)

401
